# Transfer learning with pretrained Keras models


In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser
from tqdm import tqdm
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19

from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.linear_model import LogisticRegression
from keras import backend as K
K.set_image_dim_ordering('tf')
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.models import Model
from keras.layers import Dense, Dropout, Flatten

Using TensorFlow backend.


In [2]:

start = dt.datetime.now()

# 120 Classes

In [3]:
INPUT_SIZE = 150
NUM_CLASSES = 120
SEED = 35000
data_dir = 'C:/Users/rites/Desktop/dogbreed/input/'
labels = pd.read_csv(join(data_dir, 'label.csv'))
labels_test = pd.read_csv(join(data_dir, 'sample_submission.csv'))
print(len(listdir(join(data_dir, 'train'))), len(labels))
print(len(listdir(join(data_dir, 'test'))), len(labels_test))

30801 30801
10357 10357


In [5]:
selected_breed_list = list(labels.groupby('breed').count().sort_values(by='id', ascending=False).head(NUM_CLASSES).index)
labels = labels[labels['breed'].isin(selected_breed_list)]
labels['target'] = 1
labels['rank'] = labels.groupby('breed').rank()['id']
labels_pivot = labels.pivot('id', 'breed', 'target').reset_index().fillna(0)
np.random.seed(seed=SEED)
rnd = np.random.random(len(labels))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8
y_train = labels_pivot[selected_breed_list].values
ytr = y_train[train_idx]
yv = y_train[valid_idx]


In [6]:
def read_img(img_id, train_or_test, size):
    """Read and resize image.
    # Arguments
        img_id: string
        train_or_test: string 'train' or 'test'.
        size: resize the original image.
    # Returns
        Image as numpy array.
    """
    img = image.load_img(join(data_dir, train_or_test, '%s.jpg' % img_id), target_size=size)
    img = image.img_to_array(img)
    return img

# VGG 16 Feauture Extraction

# Extract VGG16 bottleneck features

In [7]:
INPUT_SIZE = 150
POOLING = 'avg'

x_train = np.zeros((len(labels), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels['id'])):
    img = read_img(img_id, 'train', (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_train[i] = x
print('Train Images shape: {} size: {:,}'.format(x_train.shape, x_train.size))



24305it [01:43, 233.86it/s]


Train Images shape: (24305, 150, 150, 3) size: 1,640,587,500


In [8]:
INPUT_SIZE = 150
POOLING = 'avg'

x_test = np.zeros((len(labels_test), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, img_id in tqdm(enumerate(labels_test['id'])):
    img = read_img(img_id, 'test', (INPUT_SIZE, INPUT_SIZE))
    x = preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_test[i] = x
print('Test Images shape: {} size: {:,}'.format(x_test.shape, x_test.size))

10357it [00:49, 210.60it/s]


Test Images shape: (10357, 150, 150, 3) size: 699,097,500


In [9]:
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]

print((Xtr.shape, Xv.shape, ytr.shape, yv.shape))
vgg_bottleneck = VGG16(weights='imagenet', include_top=False, pooling=POOLING)
x = vgg_bottleneck.output
x = Dense(512, activation='relu', name='extra_layer_1')(x)
x = Dense(512, activation='relu', name='extra_layer_2')(x)
x = Dropout(0.5, noise_shape=None, seed=None)(x)
x = Dense(256, activation='relu', name='extra_layer_3')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=vgg_bottleneck.input, outputs=predictions)

for layer in vgg_bottleneck.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


((19415, 150, 150, 3), (4890, 150, 150, 3), (19415, 120), (4890, 120))
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block

In [10]:
model.fit(Xtr, ytr, epochs=2, validation_data=(Xv, yv), verbose=1)

Train on 19415 samples, validate on 4890 samples
Epoch 1/2


 6112/19415 [========>.....................] - ETA: 2:19:23 - loss: 14.8302 - acc: 0.0000e+0 - ETA: 2:12:53 - loss: 14.1308 - acc: 0.0156    - ETA: 2:13:52 - loss: 14.1527 - acc: 0.010 - ETA: 2:16:22 - loss: 13.8110 - acc: 0.007 - ETA: 2:15:27 - loss: 13.8746 - acc: 0.006 - ETA: 2:14:37 - loss: 13.9873 - acc: 0.005 - ETA: 2:15:05 - loss: 13.8895 - acc: 0.004 - ETA: 2:13:35 - loss: 13.6344 - acc: 0.003 - ETA: 2:11:34 - loss: 13.4773 - acc: 0.003 - ETA: 2:09:57 - loss: 13.2930 - acc: 0.003 - ETA: 2:08:54 - loss: 13.1825 - acc: 0.005 - ETA: 2:07:40 - loss: 13.0564 - acc: 0.005 - ETA: 2:06:37 - loss: 12.9883 - acc: 0.004 - ETA: 2:05:43 - loss: 12.8490 - acc: 0.004 - ETA: 2:04:56 - loss: 12.6801 - acc: 0.004 - ETA: 2:04:01 - loss: 12.4852 - acc: 0.003 - ETA: 2:03:18 - loss: 12.3215 - acc: 0.003 - ETA: 2:02:35 - loss: 12.1422 - acc: 0.003 - ETA: 2:01:59 - loss: 11.8901 - acc: 0.003 - ETA: 2:01:26 - loss: 11.6508 - acc: 0.006 - ETA: 2:00:50 - loss: 11.4810 - acc: 0.008 - ETA: 2:00:16 - loss: 

12480/19415 [==================>...........] - ETA: 1:21:27 - loss: 5.5595 - acc: 0.03 - ETA: 1:21:15 - loss: 5.5540 - acc: 0.03 - ETA: 1:21:02 - loss: 5.5479 - acc: 0.03 - ETA: 1:20:50 - loss: 5.5410 - acc: 0.03 - ETA: 1:20:38 - loss: 5.5362 - acc: 0.03 - ETA: 1:20:25 - loss: 5.5299 - acc: 0.03 - ETA: 1:20:13 - loss: 5.5223 - acc: 0.03 - ETA: 1:20:01 - loss: 5.5170 - acc: 0.03 - ETA: 1:19:48 - loss: 5.5108 - acc: 0.03 - ETA: 1:19:36 - loss: 5.5067 - acc: 0.03 - ETA: 1:19:24 - loss: 5.5023 - acc: 0.03 - ETA: 1:19:12 - loss: 5.4955 - acc: 0.03 - ETA: 1:19:00 - loss: 5.4908 - acc: 0.03 - ETA: 1:18:47 - loss: 5.4853 - acc: 0.03 - ETA: 1:18:35 - loss: 5.4797 - acc: 0.03 - ETA: 1:18:23 - loss: 5.4748 - acc: 0.03 - ETA: 1:18:10 - loss: 5.4690 - acc: 0.03 - ETA: 1:17:58 - loss: 5.4638 - acc: 0.03 - ETA: 1:17:46 - loss: 5.4579 - acc: 0.03 - ETA: 1:17:34 - loss: 5.4534 - acc: 0.03 - ETA: 1:17:22 - loss: 5.4493 - acc: 0.03 - ETA: 1:17:10 - loss: 5.4424 - acc: 0.03 - ETA: 1:16:57 - loss: 5.4380 -

19008/19415 [============================>.] - ETA: 42:02 - loss: 4.8152 - acc: 0.06 - ETA: 41:50 - loss: 4.8120 - acc: 0.06 - ETA: 41:38 - loss: 4.8092 - acc: 0.06 - ETA: 41:26 - loss: 4.8062 - acc: 0.06 - ETA: 41:15 - loss: 4.8041 - acc: 0.06 - ETA: 41:03 - loss: 4.8020 - acc: 0.06 - ETA: 40:51 - loss: 4.7993 - acc: 0.06 - ETA: 40:39 - loss: 4.7965 - acc: 0.06 - ETA: 40:27 - loss: 4.7937 - acc: 0.06 - ETA: 40:16 - loss: 4.7903 - acc: 0.06 - ETA: 40:04 - loss: 4.7875 - acc: 0.06 - ETA: 39:52 - loss: 4.7852 - acc: 0.07 - ETA: 39:40 - loss: 4.7841 - acc: 0.06 - ETA: 39:28 - loss: 4.7816 - acc: 0.07 - ETA: 39:17 - loss: 4.7802 - acc: 0.07 - ETA: 39:05 - loss: 4.7779 - acc: 0.07 - ETA: 38:53 - loss: 4.7756 - acc: 0.07 - ETA: 38:41 - loss: 4.7729 - acc: 0.07 - ETA: 38:30 - loss: 4.7705 - acc: 0.07 - ETA: 38:18 - loss: 4.7677 - acc: 0.07 - ETA: 38:06 - loss: 4.7656 - acc: 0.07 - ETA: 37:55 - loss: 4.7623 - acc: 0.07 - ETA: 37:43 - loss: 4.7603 - acc: 0.07 - ETA: 37:31 - loss: 4.7583 - acc: 

19415/19415 [==============================] - ETA: 2:16 - loss: 4.4082 - acc: 0.100 - ETA: 2:05 - loss: 4.4070 - acc: 0.100 - ETA: 1:53 - loss: 4.4054 - acc: 0.100 - ETA: 1:41 - loss: 4.4037 - acc: 0.100 - ETA: 1:30 - loss: 4.4012 - acc: 0.101 - ETA: 1:18 - loss: 4.3989 - acc: 0.101 - ETA: 1:06 - loss: 4.3963 - acc: 0.102 - ETA: 55s - loss: 4.3947 - acc: 0.102 - ETA: 43s - loss: 4.3929 - acc: 0.10 - ETA: 31s - loss: 4.3914 - acc: 0.10 - ETA: 20s - loss: 4.3900 - acc: 0.10 - ETA: 8s - loss: 4.3888 - acc: 0.1024 - 8851s 456ms/step - loss: 4.3890 - acc: 0.1024 - val_loss: 3.1999 - val_acc: 0.2319
Epoch 2/2


 6208/19415 [========>.....................] - ETA: 2:00:19 - loss: 3.4657 - acc: 0.06 - ETA: 1:58:36 - loss: 3.4131 - acc: 0.07 - ETA: 1:57:58 - loss: 3.4175 - acc: 0.12 - ETA: 1:57:13 - loss: 3.3279 - acc: 0.14 - ETA: 1:57:02 - loss: 3.3573 - acc: 0.15 - ETA: 1:56:34 - loss: 3.3867 - acc: 0.15 - ETA: 1:56:24 - loss: 3.3744 - acc: 0.14 - ETA: 1:56:02 - loss: 3.3329 - acc: 0.15 - ETA: 1:55:52 - loss: 3.3277 - acc: 0.15 - ETA: 1:55:33 - loss: 3.3326 - acc: 0.15 - ETA: 1:55:37 - loss: 3.3735 - acc: 0.15 - ETA: 1:55:19 - loss: 3.3628 - acc: 0.15 - ETA: 1:55:02 - loss: 3.3242 - acc: 0.15 - ETA: 1:55:05 - loss: 3.3283 - acc: 0.15 - ETA: 1:55:17 - loss: 3.3062 - acc: 0.16 - ETA: 1:56:05 - loss: 3.2935 - acc: 0.16 - ETA: 1:56:14 - loss: 3.2815 - acc: 0.17 - ETA: 1:56:22 - loss: 3.2767 - acc: 0.17 - ETA: 1:56:33 - loss: 3.2805 - acc: 0.17 - ETA: 1:56:41 - loss: 3.2957 - acc: 0.17 - ETA: 1:56:47 - loss: 3.2887 - acc: 0.17 - ETA: 1:56:46 - loss: 3.3053 - acc: 0.17 - ETA: 1:56:44 - loss: 3.2949 -

12576/19415 [==================>...........] - ETA: 1:21:51 - loss: 3.2565 - acc: 0.20 - ETA: 1:21:39 - loss: 3.2593 - acc: 0.20 - ETA: 1:21:26 - loss: 3.2606 - acc: 0.20 - ETA: 1:21:13 - loss: 3.2619 - acc: 0.20 - ETA: 1:21:01 - loss: 3.2611 - acc: 0.20 - ETA: 1:20:48 - loss: 3.2586 - acc: 0.20 - ETA: 1:20:35 - loss: 3.2591 - acc: 0.20 - ETA: 1:20:23 - loss: 3.2605 - acc: 0.20 - ETA: 1:20:10 - loss: 3.2610 - acc: 0.20 - ETA: 1:19:57 - loss: 3.2615 - acc: 0.20 - ETA: 1:19:44 - loss: 3.2608 - acc: 0.20 - ETA: 1:19:32 - loss: 3.2608 - acc: 0.20 - ETA: 1:19:19 - loss: 3.2605 - acc: 0.20 - ETA: 1:19:06 - loss: 3.2582 - acc: 0.20 - ETA: 1:18:53 - loss: 3.2565 - acc: 0.20 - ETA: 1:18:41 - loss: 3.2584 - acc: 0.20 - ETA: 1:18:28 - loss: 3.2574 - acc: 0.20 - ETA: 1:18:16 - loss: 3.2559 - acc: 0.20 - ETA: 1:18:03 - loss: 3.2558 - acc: 0.20 - ETA: 1:17:50 - loss: 3.2552 - acc: 0.20 - ETA: 1:17:38 - loss: 3.2523 - acc: 0.20 - ETA: 1:17:25 - loss: 3.2505 - acc: 0.20 - ETA: 1:17:13 - loss: 3.2498 -

19104/19415 [============================>.] - ETA: 41:43 - loss: 3.1819 - acc: 0.21 - ETA: 41:31 - loss: 3.1832 - acc: 0.21 - ETA: 41:19 - loss: 3.1825 - acc: 0.21 - ETA: 41:07 - loss: 3.1820 - acc: 0.21 - ETA: 40:56 - loss: 3.1816 - acc: 0.21 - ETA: 40:44 - loss: 3.1814 - acc: 0.21 - ETA: 40:32 - loss: 3.1815 - acc: 0.21 - ETA: 40:20 - loss: 3.1814 - acc: 0.21 - ETA: 40:08 - loss: 3.1809 - acc: 0.21 - ETA: 39:56 - loss: 3.1801 - acc: 0.21 - ETA: 39:44 - loss: 3.1799 - acc: 0.21 - ETA: 39:32 - loss: 3.1780 - acc: 0.21 - ETA: 39:20 - loss: 3.1775 - acc: 0.21 - ETA: 39:09 - loss: 3.1771 - acc: 0.21 - ETA: 38:57 - loss: 3.1768 - acc: 0.21 - ETA: 38:45 - loss: 3.1777 - acc: 0.21 - ETA: 38:33 - loss: 3.1769 - acc: 0.21 - ETA: 38:21 - loss: 3.1766 - acc: 0.21 - ETA: 38:09 - loss: 3.1761 - acc: 0.21 - ETA: 37:57 - loss: 3.1759 - acc: 0.21 - ETA: 37:45 - loss: 3.1743 - acc: 0.21 - ETA: 37:33 - loss: 3.1739 - acc: 0.21 - ETA: 37:22 - loss: 3.1730 - acc: 0.21 - ETA: 37:10 - loss: 3.1716 - acc: 

19415/19415 [==============================] - ETA: 1:42 - loss: 3.1276 - acc: 0.222 - ETA: 1:30 - loss: 3.1270 - acc: 0.222 - ETA: 1:18 - loss: 3.1269 - acc: 0.222 - ETA: 1:06 - loss: 3.1254 - acc: 0.223 - ETA: 55s - loss: 3.1247 - acc: 0.223 - ETA: 43s - loss: 3.1244 - acc: 0.22 - ETA: 31s - loss: 3.1239 - acc: 0.22 - ETA: 20s - loss: 3.1238 - acc: 0.22 - ETA: 8s - loss: 3.1237 - acc: 0.2233 - 8877s 457ms/step - loss: 3.1240 - acc: 0.2233 - val_loss: 2.8001 - val_acc: 0.2710


In [11]:
preds = model.predict(x_test, verbose=1)

10357/10357 [==============================] - ETA: 1:03:2 - ETA: 1:02:2 - ETA: 1:01:5 - ETA: 1:01:3 - ETA: 1:01:2 - ETA: 1:01:2 - ETA: 1:01:0 - ETA: 1:00:5 - ETA: 1:00:4 - ETA: 1:00:3 - ETA: 1:00:2 - ETA: 1:00:1 - ETA: 1:00:0 - ETA: 59:50  - ETA: 59:3 - ETA: 59:2 - ETA: 59:1 - ETA: 58:5 - ETA: 58:4 - ETA: 58:3 - ETA: 58:1 - ETA: 58:0 - ETA: 57:5 - ETA: 57:4 - ETA: 57:3 - ETA: 57:3 - ETA: 57:2 - ETA: 57:2 - ETA: 57:1 - ETA: 57:0 - ETA: 57:0 - ETA: 56:5 - ETA: 56:4 - ETA: 56:3 - ETA: 56:3 - ETA: 56:3 - ETA: 56:3 - ETA: 56:2 - ETA: 56:0 - ETA: 55:5 - ETA: 55:4 - ETA: 55:2 - ETA: 55:1 - ETA: 55:0 - ETA: 54:4 - ETA: 54:3 - ETA: 54:1 - ETA: 54:0 - ETA: 53:5 - ETA: 53:4 - ETA: 53:2 - ETA: 53:1 - ETA: 53:0 - ETA: 52:4 - ETA: 52:3 - ETA: 52:2 - ETA: 52:1 - ETA: 51:5 - ETA: 51:4 - ETA: 51:3 - ETA: 51:2 - ETA: 51:1 - ETA: 50:5 - ETA: 50:4 - ETA: 50:3 - ETA: 50:1 - ETA: 50:0 - ETA: 49:5 - ETA: 49:4 - ETA: 49:2 - ETA: 49:1 - ETA: 49:0 - ETA: 48:5 - ETA: 48:4 - ETA: 48:2 - ETA: 48:1 - ETA: 48:0 - E

In [12]:
targets_series = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [13]:
sub = pd.DataFrame(preds)
# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
sub.insert(0, 'id', labels_test['id'])
sub.head(5)

id  Afghan_hound  African_hunting_dog  \
0  000621fb3cbb32d8935728e48679680e      0.044146             0.000836   
1  00102ee9d8eb90812350685311fe5890      0.000053             0.002142   
2  0012a730dfa437f5f3613fb75efcd4ce      0.005891             0.009408   
3  001510bc8570bbeee98c8d80c8a95ec1      0.006861             0.017898   
4  001a5f3114548acdefa3d4da05474c2e      0.063607             0.016154   

       Airedale  American_Staffordshire_terrier  Appenzeller  \
0  1.491983e-03                        0.025924     0.031389   
1  4.876950e-07                        0.001794     0.000006   
2  5.483827e-03                        0.004339     0.004178   
3  7.950988e-03                        0.016219     0.004661   
4  5.255191e-05                        0.004238     0.001516   

   Australian_terrier  Bedlington_terrier  Bernese_mountain_dog  \
0            0.001793            0.007022              0.000778   
1            0.000016            0.000054              0.000005   
2            0.003435            0.003224              0.006300   
3            0.004465            0.005952              0.013875   
4            0.050634            0.000908              0.000330   

   Blenheim_spaniel           ...             schipperke  silky_terrier  \
0          0.000100           ...               0.000984       0.006930   
1          0.000065           ...               0.000034       0.003036   
2          0.002957           ...               0.019221       0.002095   
3          0.004067           ...               0.004622       0.004725   
4          0.003061           ...               0.000522       0.000324   

   soft-coated_wheaten_terrier  standard_poodle  standard_schnauzer  \
0                     0.000143         0.020064        6.449800e-03   
1                     0.000013         0.000136        9.788290e-07   
2                     0.002253         0.003296        5.883866e-03   
3                     0.012811         0.001946        3.623846e-03   
4                     0.000124         0.003534        7.163270e-02   

   toy_poodle  toy_terrier    vizsla   whippet  wire-haired_fox_terrier  
0    0.000077     0.000434  0.001953  0.001831             4.572930e-06  
1    0.000027     0.000311  0.000005  0.006817             4.646315e-07  
2    0.004804     0.004247  0.010778  0.001348             5.693787e-03  
3    0.012221     0.001624  0.000949  0.002946             1.911383e-03  
4    0.000356     0.010759  0.030712  0.000181             1.421953e-03  

[5 rows x 121 columns]

In [19]:
sub.to_csv("C:/Users/rites/Desktop/dogbreed/sample_submission.csv")